<span style="font-size:24px; font-family:'Roboto'; font-weight:bold;">
Script to visualize iMOD inputs in QGIS.
</span><br>

In [1]:
print(f"\033[1m{'*'*50} Mdl_In_to_mm {'*'*50}\033[0m")
print('This script converts model inputs (mainly IDF) to TIF files, to be visualized in QGIS. It also calculates some parameters from Mdl Ins (e.g.) Aquifer layer thickness is calculated from TOP-BOT (same as MF does internally).')

************************************************** Mdl_In_to_mm **************************************************
This script converts model inputs (mainly IDF) to TIF files, to be visualized in QGIS. It also calculates some parameters from Mdl Ins (e.g.) Aquifer layer thickness is calculated from TOP-BOT (same as MF does internally).


## 0.0. Libraries

In [2]:
from WS_Mdl import utils as U
from WS_Mdl import geo as G
from WS_Mdl import utils_imod as UIM

In [3]:
import importlib
importlib.reload(G)
importlib.reload(U)
importlib.reload(UIM)

<module 'WS_Mdl.utils_imod' from 'c:\\users\\karam014\\onedrive - universiteit utrecht\\ws_mdl\\code\\WS_Mdl\\utils_imod.py'>

## 0.1. Options

In [4]:
MdlN = 'NBr13'

# Process Regular, Time Independent IDF Files

Select rows to be converted.

In [5]:
# G.PRJ_to_TIF(MdlN)

---

In [6]:
DF = UIM.PRJ_to_DF(MdlN, verbose=False)

✅ - NBr13 paths extracted from RunLog.


In [7]:
DF

,package,parameter,MdlN,time,active,is_constant,layer,factor,addition,constant,suffix,path
(CAP)_boundary,CAP,boundary,NBr1,NaN,True,2,1,1.0,0.0,-9999.0,.idf,C:/OD/WS_Mdl\models\NBr\In\BND\BND_L1_NBr1.IDF
(CAP)_landuse,CAP,landuse,NBr1,NaN,True,2,1,1.0,0.0,100000000000000000000.0,.idf,C:/OD/WS_Mdl\models\NBr\In\CAP\LGN\LGN_NBr1.idf
(CAP)_rootzone_thickness,CAP,rootzone_thickness,NBr1,NaN,True,2,1,1.0,0.0,100000000000000000000.0,.idf,C:/OD/WS_Mdl\models\NBr\In\CAP\RZN\RZN_NBr1.idf
(CAP)_soil_physical_unit,CAP,soil_physical_unit,NBr1,NaN,True,2,1,1.0,0.0,100000000000000000000.0,.idf,C:/OD/WS_Mdl\models\NBr\In\CAP\SFU\SFU_NBr1.idf
(CAP)_meteo_station_number,CAP,meteo_station_number,NBr1,NaN,True,2,1,1.0,0.0,-9999.0,.idf,C:/OD/WS_Mdl\models\NBr\In\BND\BND_L1_NBr1.IDF
...,...,...,...,...,...,...,...,...,...,...,...,...
(CHD)_head_Sys17_2018-12-14 00:00:00,CHD,head,NBr5,2018-12-14 00:00:00,True,2,33,1.0,0.0,-999.99,.idf,C:/OD/WS_Mdl\models\NBr\In\CHD\NBr5\head_20181...
(CHD)_head_Sys18_2018-12-14 00:00:00,CHD,head,NBr5,2018-12-14 00:00:00,True,2,35,1.0,0.0,-999.99,.idf,C:/OD/WS_Mdl\models\NBr\In\CHD\NBr5\head_20181...
(CHD)_head_Sys19_2018-12-14 00:00:00,CHD,head,NBr5,2018-12-14 00:00:00,True,2,37,1.0,0.0,-999.99,.idf,C:/OD/WS_Mdl\models\NBr\In\CHD\NBr5\head_20181...
PERIODS,-,-,-,-,Failed to read package,-,-,-,-,-,-,-


In [8]:
d_paths = U.get_MdlN_paths(MdlN)

✅ - NBr13 paths extracted from RunLog.


In [9]:
MdlN_B = d_paths['MdlN_B']

In [10]:
DF_B = UIM.PRJ_to_DF(MdlN_B, verbose=False)

✅ - NBr12 paths extracted from RunLog.


In [11]:
DF_replace = DF.merge(DF_B, how='outer', indicator=True).query('_merge != "both"')

In [30]:
d_replace = {}

In [12]:
DF_replace

,package,parameter,MdlN,time,active,is_constant,layer,factor,addition,constant,suffix,path,_merge
2403,RIV,bottom_elevation,NBr1,steady-state,True,2,0,1.0,0.0,-999.99,.idf,C:/OD/WS_Mdl\models\NBr\In\RIV\RIV_Stg_Detailw...,right_only
2404,RIV,bottom_elevation,NBr1,steady-state,True,2,0,1.0,0.0,-999.99,.idf,C:/OD/WS_Mdl\models\NBr\In\RIV\RIV_Stg_winter_...,right_only
2408,RIV,bottom_elevation,NBr1,summer,True,2,0,1.0,0.0,-999.99,.idf,C:/OD/WS_Mdl\models\NBr\In\RIV\RIV_Stg_Detailw...,right_only
2409,RIV,bottom_elevation,NBr1,summer,True,2,0,1.0,0.0,-999.99,.idf,C:/OD/WS_Mdl\models\NBr\In\RIV\RIV_Stg_summer_...,right_only
2413,RIV,bottom_elevation,NBr1,winter,True,2,0,1.0,0.0,-999.99,.idf,C:/OD/WS_Mdl\models\NBr\In\RIV\RIV_Stg_Detailw...,right_only
2414,RIV,bottom_elevation,NBr1,winter,True,2,0,1.0,0.0,-999.99,.idf,C:/OD/WS_Mdl\models\NBr\In\RIV\RIV_Stg_winter_...,right_only
2415,RIV,bottom_elevation,NBr13,steady-state,True,2,0,1.0,0.0,-999.99,.idf,C:/OD/WS_Mdl\models\NBr\In\RIV\NBr13\RIV_Stg_D...,left_only
2416,RIV,bottom_elevation,NBr13,steady-state,True,2,0,1.0,0.0,-999.99,.idf,C:/OD/WS_Mdl\models\NBr\In\RIV\NBr13\RIV_Stg_w...,left_only
2417,RIV,bottom_elevation,NBr13,summer,True,2,0,1.0,0.0,-999.99,.idf,C:/OD/WS_Mdl\models\NBr\In\RIV\NBr13\RIV_Stg_D...,left_only
2418,RIV,bottom_elevation,NBr13,summer,True,2,0,1.0,0.0,-999.99,.idf,C:/OD/WS_Mdl\models\NBr\In\RIV\NBr13\RIV_Stg_s...,left_only


In [ ]:
DF_l = DF_replace.query('_merge == "left_only"')
DF_r = DF_replace.query('_merge == "right_only"')

In [23]:
DF_replace.columns

Index(['package', 'parameter', 'MdlN', 'time', 'active', 'is_constant',
       'layer', 'factor', 'addition', 'constant', 'suffix', 'path', '_merge'],
      dtype='object')

In [26]:
Cols = DF_replace.columns.drop(['MdlN', 'path', '_merge'])
Cols

Index(['package', 'parameter', 'time', 'active', 'is_constant', 'layer',
       'factor', 'addition', 'constant', 'suffix'],
      dtype='object')

In [ ]:
for i, R_L in DF_l.iterrows():
    for i, R_R in DF_r.iterrows():
        if (R_L[Cols] == R_R[Cols]).all():
            d_replace[R_R['path']] = R_L['path']

In [32]:
d_replace

{'C:/OD/WS_Mdl\\models\\NBr\\In\\RIV\\NBr13\\RIV_Stg_Detailwatergangen_NBr13.IDF': 'C:/OD/WS_Mdl\\models\\NBr\\In\\RIV\\RIV_Stg_winter_NBr1.IDF',
 'C:/OD/WS_Mdl\\models\\NBr\\In\\RIV\\NBr13\\RIV_Stg_winter_NBr13.IDF': 'C:/OD/WS_Mdl\\models\\NBr\\In\\RIV\\RIV_Stg_winter_NBr1.IDF',
 'C:/OD/WS_Mdl\\models\\NBr\\In\\RIV\\NBr13\\RIV_Stg_summer_NBr13.IDF': 'C:/OD/WS_Mdl\\models\\NBr\\In\\RIV\\RIV_Stg_summer_NBr1.IDF',
 'C:/OD/WS_Mdl\\models\\NBr\\In\\RIV\\NBr13\\RIV_Stg_Maas_NBr13.IDF': 'C:/OD/WS_Mdl\\models\\NBr\\In\\RIV\\RIV_Stg_winter_NBr1.IDF',
 'C:/OD/WS_Mdl\\models\\NBr\\In\\RIV\\NBr13\\RIV_Stg_Polderpeil_summer_NBr13.IDF': 'C:/OD/WS_Mdl\\models\\NBr\\In\\RIV\\RIV_Stg_summer_NBr1.IDF'}